# Generate real embeddings using Mistral API's `codestral-embed` model.

In [ ]:
!pip install mistralai

In [ ]:
from mistralai import Mistral
import os

os.environ["MISTRAL_API_KEY"] = "<your_mistral_api_key>" # Replace with your actual Mistral API key

client = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))

texts = [
    "Apple is a fruit.",
    "Bananas are yellow.",
    "Cherries grow on trees."
]
response = client.embeddings.create(model="mistral-embed", inputs=texts)
vectors = [item.embedding for item in response.data]

# Store and search Mistral-generated embeddings using Weaviate vector database.

## ✅ Step-by-Step: Create Your Free Weaviate Cluster

1. Go to: [https://console.weaviate.cloud](https://console.weaviate.cloud)
2. Log in with GitHub or email
3. Click **"Create Cluster"**
4. Choose:
   - **Tier**: `Sandbox`
   - **Vectorizer**: `none`
   - **Name**: `mistral-demo`
5. Wait 1–2 minutes for the cluster to be ready
6. Click into your cluster and copy:
   - The **URL** (e.g. `https://mistral-demo.weaviate.network`)
   - Your **API key** (if authentication is enabled)

In [ ]:
!pip. install weaviate-client

In [ ]:
import weaviate
import uuid

# Setup Weaviate Cloud (replace with your real URL/API key)
WEAVIATE_URL = "https://your-cluster.weaviate.network"
WEAVIATE_API_KEY = None  # Or use your actual API key if enabled

auth_config = weaviate.auth.AuthApiKey(api_key=WEAVIATE_API_KEY) if WEAVIATE_API_KEY else None
client_w = weaviate.Client(
    url=WEAVIATE_URL,
    auth_client_secret=auth_config
)

# Define schema
class_def = {
    "class": "Doc",
    "vectorizer": "none",
    "properties": [{"name": "text", "dataType": ["text"]}]
}
if not client_w.schema.contains({"classes": [class_def]}):
    client_w.schema.create_class(class_def)

# Insert data
for text, vec in zip(texts, vectors):
    client_w.data_object.create({"text": text}, vector=vec, class_name="Doc", uuid=str(uuid.uuid4()))

# Search
query = "Yellow fruit"
q_vec = client.embeddings.create(model="codestral-embed", inputs=[query]).data[0].embedding

res = client_w.query.get("Doc", ["text"]).with_near_vector({"vector": q_vec}).with_limit(2).do()
print(res["data"]["Get"]["Doc"])

# Insert and search Mistral embeddings in Milvus vector database using pymilvus.

## ✅ Step-by-Step: Create Your Free Zilliz Cloud Cluster

1. Go to: [https://zilliz.com/cloud](https://zilliz.com/cloud)
2. Sign up or log in
3. Click **"Create Cluster"**
   - Choose **Serverless**
   - Select **Region**, name your cluster (e.g., `mistral-demo`)
   - Choose the **Free Tier** (sandbox)
4. Wait 1–2 minutes for provisioning
5. Once running, click the cluster to get:
   - `API Endpoint`: e.g., `https://xxxx.api.zillizcloud.com`
   - `API Key`: copy the access token

In [ ]:
!pip install pymilvus

In [ ]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection
from mistralai import Mistral
import os


# Connect to Zilliz Serverless
connections.connect(
    uri="https://your-cluster-name.api.zillizcloud.com",
    token="your-api-key"
)

# Define collection schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=len(vectors[0]))
]
schema = CollectionSchema(fields, description="Mistral Embeddings Example")
collection_name = "mistral_demo"

# Create collection
if collection_name not in [c.name for c in Collection.list()]:
    col = Collection(name=collection_name, schema=schema)
    col.create_index("embedding", {"index_type": "IVF_FLAT", "metric_type": "L2", "params": {"nlist": 16}})
    col.load()
else:
    col = Collection(collection_name)
    col.load()

# Insert vectors
ids = list(range(len(vectors)))
col.insert([ids, vectors])

# Query
query = "Yellow fruit"
query_vec = client.embeddings.create(model="codestral-embed", inputs=[query]).data[0].embedding

results = col.search(
    data=[query_vec],
    anns_field="embedding",
    param={"metric_type": "L2", "params": {"nprobe": 10}},
    limit=2
)

# Print results
for hits in results:
    for hit in hits:
        print(f"Match ID: {hit.id}, distance: {hit.distance:.4f}")